In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
class Human:
    def __init__(self, personid, xi, yi):
      self.i = personid
      self.x = xi
      self.y = yi
      self.path = []
      self.done = False
      self.state = '0'
      self.direction = None
    def getPersonid(self):
        return self.i
    def getX(self):
        return self.x
    def getY(self):
        return self.y
    def getPath(self):
        return self.path
    def updatePath(self, xn, yn):
        self.path.append([self.x,self.y])
        self.x = xn
        self.y = yn
    def getState(self):
        return self.state
    def setState(self, statex):
        self.state=statex
    def getDir(self):
        return self.direction
    def setDone(self):
        self.done = True
    def isdone(self):
        return self.done
    def movingout(self,mid_start,mid_end):
        if len(self.path) >= 2:
            if self.state == '0':
                if self.path[-1][1] < mid_end and self.path[-2][1] >= mid_end:
                    self.state = '1'
                    self.direction = 'up'
                    return True
            else:
                return False
        else:
            return False
    def movingin(self,mid_start,mid_end):
        if len(self.path) >= 2:
            if self.state == '0':
                if self.path[-1][1] > mid_start and self.path[-2][1] <= mid_start:
                    self.state = '1'
                    self.direction = 'down'
                    return True
            else:
                return False
        else:
            return False

In [0]:
import numpy as np
import cv2

cap = cv2.VideoCapture("/content/drive/My Drive/myvid.avi")
res=(int(cap.get(3)),int(cap.get(4)))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output1.avi',fourcc, 20.0, res)
w = cap.get(3)
h = cap.get(4)
minarea = (h*w)/300

line_a = int(1*(h/5))
line_a2 = int(4*(h/5))
line_b = int(.5*(h/5))
line_b2 = int(4.5*(h/5))
line1 = np.array([[0, line_a2],[w, line_a2]], np.int32)
line1 = line1.reshape((-1,1,2))
line2 = np.array([[0, line_a],[w, line_a]], np.int32)
line2 = line2.reshape((-1,1,2))
line3 = np.array([[0, line_b],[w, line_b]], np.int32)
line3 = line3.reshape((-1,1,2))
line4 = np.array([[0, line_b2],[w, line_b2]], np.int32)
line4 = line4.reshape((-1,1,2))
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows = True)
kernelOp = np.ones((3,3),np.uint8)
kernelOp2 = np.ones((5,5),np.uint8)
kernelCl = np.ones((11,11),np.uint8)
humans = []
human_no = 1
outside = 0
inside = 0

while(True):
    
    ret, img = cap.read()
    if(ret==False):
        print ('Went out:',outside)
        print ('Went in:',inside)
        print ('Total Inside:',(inside-outside))
        break

    fgmask = fgbg.apply(img)
    fgmask2 = fgbg.apply(img)
    ret,imBin= cv2.threshold(fgmask,200,255,cv2.THRESH_BINARY)
    ret,imBin2 = cv2.threshold(fgmask2,200,255,cv2.THRESH_BINARY)
    mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN, kernelOp)
    mask2 = cv2.morphologyEx(imBin2, cv2.MORPH_OPEN, kernelOp)
    mask =  cv2.morphologyEx(mask , cv2.MORPH_CLOSE, kernelCl)
    mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernelCl)
    allcontours, hierarchy = cv2.findContours(mask2,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for contour in allcontours:
        rect = cv2.boundingRect(contour)
        area = cv2.contourArea(contour)
        if area > minarea:
            M = cv2.moments(contour)
            centx = int(M['m10']/M['m00'])
            centy = int(M['m01']/M['m00'])
            x,y,w,h = cv2.boundingRect(contour)
            new = True
            if centy in range(line_b,line_b2):
                for i in humans:
                    if abs(centx-i.getX()) <= w and abs(centy-i.getY()) <= h:
                        new = False
                        i.updatePath(centx,centy)
                        if i.movingout(line_a2,line_a) == True:
                            outside += 1;
                        elif i.movingin(line_a2,line_a) == True:
                            inside += 1;
                        break
                    if i.getState() == '1':
                        if i.getDir() == 'down' and i.getY() > line_b2:
                            i.setDone()
                        elif i.getDir() == 'up' and i.getY() < line_b:
                            i.setDone()
                    if i.isdone():
                        index = humans.index(i)
                        humans.pop(index)
                        del i
                if new == True:
                    h = Human(human_no,centx,centy)
                    humans.append(h)
                    human_no += 1         
            cv2.circle(img,(centx,centy), 3, (0,255,0), -1)         
            
    string1 = 'WENT OUT: '+ str(outside)
    string2 = 'WENT IN: '+ str(inside)
    string3 = 'CURRENTLY INSIDE: '+ str(inside-outside)
    img = cv2.polylines(img,[line1],False,(0,0,255),thickness=2)
    img = cv2.polylines(img,[line2],False,(0,0,255),thickness=2)
    cv2.putText(img, string1 ,(10,10),cv2.FONT_HERSHEY_SIMPLEX,0.3,(255,255,255),1,cv2.LINE_AA)
    cv2.putText(img, string2 ,(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.3,(255,255,255),1,cv2.LINE_AA)
    cv2.putText(img, string3 ,(10,50),cv2.FONT_HERSHEY_SIMPLEX,0.3,(255,255,255),1,cv2.LINE_AA)
    out.write(img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
cap.release()
out.release()